In [1]:
import pandas as pd 
import numpy as np

## Rolling Join

In [2]:
groups = pd.DataFrame({
    "group":["A", "B", "C", "D", "E", "F", "G", "H", "I", "J",],
    "cdf":[.0, .1, .2, .3, .4, .5, .6, .7, .8, .9,],
    }).sort_values("cdf", ascending=True).reset_index(drop=True)

observations = pd.DataFrame({
    "id":[str(i).zfill(5) for i in range(25)],
    "cdf":np.random.rand(25),
    }).sort_values("cdf", ascending=True).reset_index(drop=True)

In [5]:
groups

,group,cdf
0,A,0.0
1,B,0.1
2,C,0.2
3,D,0.3
4,E,0.4
5,F,0.5
6,G,0.6
7,H,0.7
8,I,0.8
9,J,0.9


In [6]:
observations

,id,cdf
0,00014,0.033505
1,00009,0.037458
2,00004,0.128786
3,00019,0.246572
4,00002,0.247654
5,00001,0.287682
6,00022,0.406659
7,00018,0.451544
8,00024,0.468223
9,00020,0.547750


Assume: in `observations` are subjects, in `groups` are predefined groups with the lower bound cdf. 

Objective: specify the subjects into the corresponding group, such that cdf in `groups` is: 
1. <= cdf in `observations`
2. the largest one when (1) satisfied

In [7]:
pd.merge_asof(observations, groups, on='cdf', direction='backward')

,id,cdf,group
0,00014,0.033505,A
1,00009,0.037458,A
2,00004,0.128786,B
3,00019,0.246572,C
4,00002,0.247654,C
5,00001,0.287682,C
6,00022,0.406659,E
7,00018,0.451544,E
8,00024,0.468223,E
9,00020,0.547750,F


## Window Function

### Rolling

In [30]:
df = pd.DataFrame(np.random.randn(8, 4),index = pd.date_range('12/1/2020', periods=8),columns = ['A', 'B', 'C', 'D'])
df

,A,B,C,D
2020-12-01,-1.858614,2.648502,0.447054,-0.567699
2020-12-02,0.217183,0.782849,0.340099,-0.075707
2020-12-03,-1.403748,-0.724513,-1.361126,0.081721
2020-12-04,-1.702775,-1.043019,-0.578377,-1.700636
2020-12-05,0.095660,-0.333369,-1.404761,1.424716
2020-12-06,-0.082435,1.028414,-0.880428,-0.227474
2020-12-07,0.862362,-1.512562,0.669662,-0.334483
2020-12-08,-1.209254,-1.315626,-1.413272,-0.055986


In [31]:
# rolling 
df.rolling(window=3).mean()

,A,B,C,D
2020-12-01,NaN,NaN,NaN,NaN
2020-12-02,NaN,NaN,NaN,NaN
2020-12-03,-1.015060,0.902279,-0.191324,-0.187228
2020-12-04,-0.963113,-0.328228,-0.533135,-0.564874
2020-12-05,-1.003621,-0.700300,-1.114755,-0.064733
2020-12-06,-0.563183,-0.115991,-0.954522,-0.167798
2020-12-07,0.291862,-0.272506,-0.538509,0.287586
2020-12-08,-0.143109,-0.599925,-0.541346,-0.205981


In [33]:
df.rolling(window=3, min_periods=1).sum()

,A,B,C,D
2020-12-01,-1.858614,2.648502,0.447054,-0.567699
2020-12-02,-1.641431,3.431352,0.787153,-0.643406
2020-12-03,-3.045179,2.706838,-0.573973,-0.561685
2020-12-04,-2.889340,-0.984683,-1.599404,-1.694622
2020-12-05,-3.010863,-2.100901,-3.344264,-0.194199
2020-12-06,-1.689550,-0.347974,-2.863565,-0.503394
2020-12-07,0.875587,-0.817517,-1.615527,0.862759
2020-12-08,-0.429327,-1.799774,-1.624038,-0.617943


### Aggregate

In [35]:
company=["A","B","C"]
data=pd.DataFrame({
    "dept":[company[x] for x in np.random.randint(0,len(company),8)],
    "name":["a","b","c","d","e","f","g","h"],
    "salary":np.random.randint(10,30,8)
}
)
data

,dept,name,salary
0,B,a,10
1,C,b,29
2,A,c,19
3,B,d,15
4,A,e,12
5,A,f,28
6,B,g,15
7,C,h,15


In [45]:
data['sum_salary'] = data.groupby('dept')['salary'].transform('sum')
data

,dept,name,salary,sum_salary,rank,row_number,dense_rank
0,B,a,10,40,3.0,3.0,2.0
1,C,b,29,44,1.0,1.0,1.0
2,A,c,19,59,2.0,2.0,2.0
3,B,d,15,40,1.0,1.0,1.0
4,A,e,12,59,3.0,3.0,3.0
5,A,f,28,59,1.0,1.0,1.0
6,B,g,15,40,1.0,2.0,1.0
7,C,h,15,44,2.0,2.0,2.0


### Rank

In [46]:
data['rank'] = data.groupby('dept')['salary'].rank(ascending=False, method='min')
data['row_number'] = data.groupby('dept')['salary'].rank(ascending=False, method='first') 
data['dense_rank'] = data.groupby('dept')['salary'].rank(ascending=False, method='dense')
data

,dept,name,salary,sum_salary,rank,row_number,dense_rank
0,B,a,10,40,3.0,3.0,2.0
1,C,b,29,44,1.0,1.0,1.0
2,A,c,19,59,2.0,2.0,2.0
3,B,d,15,40,1.0,1.0,1.0
4,A,e,12,59,3.0,3.0,3.0
5,A,f,28,59,1.0,1.0,1.0
6,B,g,15,40,1.0,2.0,1.0
7,C,h,15,44,2.0,2.0,2.0


In [50]:
data['percent_rank'] = data.groupby('dept')['salary'].rank(ascending=False, method='min', pct=True) 
data['cum_dist'] = data.groupby('dept')['salary'].rank(ascending=False, method='first', pct=True)
data

,dept,name,salary,sum_salary,rank,row_number,dense_rank,percent_rank,cum_dist
0,B,a,10,40,3.0,3.0,2.0,1.000000,1.000000
1,C,b,29,44,1.0,1.0,1.0,0.500000,0.500000
2,A,c,19,59,2.0,2.0,2.0,0.666667,0.666667
3,B,d,15,40,1.0,1.0,1.0,0.333333,0.333333
4,A,e,12,59,3.0,3.0,3.0,1.000000,1.000000
5,A,f,28,59,1.0,1.0,1.0,0.333333,0.333333
6,B,g,15,40,1.0,2.0,1.0,0.333333,0.666667
7,C,h,15,44,2.0,2.0,2.0,1.000000,1.000000


In [51]:
data[data.dept=='B']

,dept,name,salary,sum_salary,rank,row_number,dense_rank,percent_rank,cum_dist
0,B,a,10,40,3.0,3.0,2.0,1.000000,1.000000
3,B,d,15,40,1.0,1.0,1.0,0.333333,0.333333
6,B,g,15,40,1.0,2.0,1.0,0.333333,0.666667


### Shift

In [56]:
data_sorted = data.sort_values(['dept', 'salary'], ascending=False)
data_sorted

,dept,name,salary,sum_salary,rank,row_number,dense_rank,percent_rank,cum_dist
1,C,b,29,44,1.0,1.0,1.0,0.500000,0.500000
7,C,h,15,44,2.0,2.0,2.0,1.000000,1.000000
3,B,d,15,40,1.0,1.0,1.0,0.333333,0.333333
6,B,g,15,40,1.0,2.0,1.0,0.333333,0.666667
0,B,a,10,40,3.0,3.0,2.0,1.000000,1.000000
5,A,f,28,59,1.0,1.0,1.0,0.333333,0.333333
2,A,c,19,59,2.0,2.0,2.0,0.666667,0.666667
4,A,e,12,59,3.0,3.0,3.0,1.000000,1.000000


In [60]:
data_sorted['lead'] = data_sorted.groupby(['dept'])['salary'].shift(-1)
data_sorted['lag'] = data_sorted.groupby(['dept'])['salary'].shift(1)
data_sorted

,dept,name,salary,sum_salary,rank,row_number,dense_rank,percent_rank,cum_dist,lead,lag
1,C,b,29,44,1.0,1.0,1.0,0.500000,0.500000,15.0,NaN
7,C,h,15,44,2.0,2.0,2.0,1.000000,1.000000,NaN,29.0
3,B,d,15,40,1.0,1.0,1.0,0.333333,0.333333,15.0,NaN
6,B,g,15,40,1.0,2.0,1.0,0.333333,0.666667,10.0,15.0
0,B,a,10,40,3.0,3.0,2.0,1.000000,1.000000,NaN,15.0
5,A,f,28,59,1.0,1.0,1.0,0.333333,0.333333,19.0,NaN
2,A,c,19,59,2.0,2.0,2.0,0.666667,0.666667,12.0,28.0
4,A,e,12,59,3.0,3.0,3.0,1.000000,1.000000,NaN,19.0


## Pivot

In [61]:
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                         "bar", "bar", "bar", "bar"],
                   "B": ["one", "one", "one", "two", "two",
                         "one", "one", "two", "two"],
                   "C": ["small", "large", "large", "small",
                         "small", "large", "small", "small",
                         "large"],
                   "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
df

,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,one,large,2,5
3,foo,two,small,3,5
4,foo,two,small,3,6
5,bar,one,large,4,6
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


In [64]:
df.pivot_table(values=['D','E'], index=['A','B'], columns=['C'], aggfunc={'D':np.sum, 'E':np.mean})

D           E      
C       large small large small
A   B                          
bar one   4.0   5.0   6.0   8.0
    two   7.0   6.0   9.0   9.0
foo one   4.0   1.0   4.5   2.0
    two   NaN   6.0   NaN   5.5